In [1]:
import os
import sys
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import pandas as pd
from functions import dirs, readFile
from wordEmbedders import WESCScore
import pickle
import chevron
from re import sub
from latexTable import LatexTable

In [2]:
datasets = dirs('../data')
canonicalNames = []
results = []
for dataset in datasets:
    name = readFile(f'../data/{dataset}/CanonicalName.txt')
    canonicalNames.append(name)
    result = WESCScore.load(f'../data/{dataset}/Word2Vec-Prediction.csv')
    results.append(result)

In [3]:
def latexMatrix(res):
    tp = str(res.truePos)
    fp = str(res.falsePos)
    fn = str(res.falseNeg)
    tn = str(res.trueNeg)
    return "\\begin{tabular}{c|c} "+tp+" & "+fp+" \\\\ \hline "+fn+" & "+tn+" \end{tabular}"

In [4]:
accuracy = list(map(lambda res: f"{res.accuracy:.3f}", results))
balanced = list(map(lambda res: f"{res.balancedAccuracy:.3f}", results))
matrixes = list(map(lambda res: latexMatrix(res), results))

In [12]:
table = LatexTable()
table.headers = canonicalNames
table.columnAlignments = 'c'
table.emptyFirstHeader = True
table.rows = [
    ['Balanced accuracy'] + accuracy,
    ['Confusion matrix'] + matrixes
]
result = table.render()
outputFile = 'Results-Accuracy.tex'
with open(outputFile, mode='w') as output:
    output.write(result)
print(result)

\begin{tabular}{|l|c|c|} \cline{2-3}
\multicolumn{1}{c|}{} & \textbf{Airline tweets} & \textbf{IMDB reviews} \\ \hline
\textbf{Balanced accuracy} & 0.531 & 0.628 \\ \hline
\textbf{Confusion matrix} & \begin{tabular}{c|c} 1165 & 3819 \\ \hline 350 & 3563 \end{tabular} & \begin{tabular}{c|c} 7206 & 825 \\ \hline 17794 & 24175 \end{tabular} \\ \hline
\end{tabular}


In [5]:
data = {
    'datasets': canonicalNames,
    'accuracy': accuracy,
    'baccuracy': balanced,
    'cmatrix': matrixes
}

In [6]:
data

{'datasets': ['Airline tweets', 'IMDB reviews'],
 'accuracy': ['0.531', '0.628'],
 'baccuracy': ['0.626', '0.628'],
 'cmatrix': ['\\begin{tabular}{c|c} 1165 & 3819 \\\\ \\hline 350 & 3563 \\end{tabular}',
  '\\begin{tabular}{c|c} 7206 & 825 \\\\ \\hline 17794 & 24175 \\end{tabular}']}

In [45]:
outputFile = 'Results-Accuracy.tex'
with open(outputFile, mode='w') as output:
    with open(f'{outputFile}.mustache') as template:
        output.write(chevron.render(template, data))

In [51]:
rows = []
for i in range(len(datasets)):
    rows.append({
        'name': canonicalNames[i],
        'data': [balanced[i], matrixes[i]]
    })
data2 = {
    'headers': ['Balanced Accuracy', 'Confusion Matrix'],
    'rows': rows
}

In [52]:
outputFile = 'Results-Accuracy-Variant.tex'
with open(outputFile, mode='w') as output:
    with open(f'{outputFile}.mustache') as template:
        output.write(chevron.render(template, data2))